In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("DABP_Raw_data.csv")

df = df.rename(columns={"eDispatch_01": "Disp_Complaint_Reported", "eDispatch_02": "EMD_Performed",
                        "eDisposition_12":"Patient_Disposition","eDisposition_19":"Final_Patient_Acuity",
                        "eDisposition_16":"Transport_Mode","eDisposition_21":"Destination",
                        "eDisposition_22":"Hospital_Inpatient_Destination",
                        "eOutcome_01":"Emergency_Department_Disposition","eOutcome_02":"Hospital_Disposition",
                       "ePatient_13":"Gender","ePatient_15":"Age","ePatient_16":"Age_Units",
                        "ePayment_50":"CMS_Service_Level","eResponse_05":"Response_Type_Service_Requested",
                       "eResponse_23":"Response_Mode_to_Scene","eScene_01":"First_EMS_Unit_Scene",
                        "eScene_06":"Number_of_Patients_Scene","eScene_07":"Mass_Casualty_Incident",
                       "eScene_08":"Triage_Classification_MCI","eScene_09":"Incident_Location_Type",
                       "eSituation_02":"Possible_Injury","'eSituation_07'":"Chief_Complaint_Anatomic_Location",
                       "'eSituation_08'":"Chief_Complaint_Organ_System","eSituation_13":"Initial_Patient_Acuity",
                       "'eSituation_01'":"Date/Time_of_Symptom_Onset","eTimes_01":"PSAP_Call_Date/Time",
                       "eTimes_03":"Unit_Notified_Dispatch_Date/Time","eTimes_05":"Unit_En_Route_Date/Time",
                       "eTimes_06":"Unit_Arrived on Scene Date/Time","eTimes_07":"Arrived_at_Patient_Date/Time",
                       "eTimes_09":"Unit_Left_Scene_Date/Time","eTimes_11":"Patient_Arrived_at_Destination_Date/Time",
                       "eTimes_12":"Destination_Patient_Transfer_of_Care_Date/Time",
                        "eTimes_13":"Unit Back in Service Date/Time","eDisposition_17":"Transport_Mode_from_Scene",
                       "CensusRegion":"Census_Region","CensusDivision":"Census_Division",
                       "'Urbanicity'":"Urbanicity","eHistory_17":"Alcohol/Drug_Use_Indicators","eInjury_01":"Cause_of_Injury",
                       "eDisposition_20":"Reason_for_Choosing_Destination","eDisposition_25":"Date/Time_of_Destination_Prearrival_Alert",
                       "eDisposition_24":"Destination_Team_Pre-Arrival_Alert","eResponse_08":"Type_of_Dispatch_Delay",
                       "eInjury_04":"Vehicular_Pedestrian_Injury_Risk_Factor","eSituation_09":"Primary_Symptom","eResponse_09":"Type_of_Response_Delay",
                       "eResponse_10":"Type_of_Scene_Delay","eResponse_11":"Type_of_Transport_Delay","eVitals_01":"Date/Time_Vital_Signs_Taken",
                       "eVitals_06":"Systolic_Blood_Pressure","eVitals_10":"Heart_Rate","eVitals_12":"Pulse_Oximetry","eVitals_14":"Respiratory_Rate",
                       "eVitals_16":"End_Tidal_Carbon_Dioxide","eVitals_18":"Blood_Glucose_Level","eVitals_27":"Pain_Scale_Score",
                       "eVitals_02":"Obtained_Prior_to_this_Unit_EMS_Care","eVitals_04":"ECG_Type","eVitals_08":"Method_of_Blood_Pressure_Measurement",
                       "eVitals_26":"Level_of_Responsiveness","eVitals_29":"Stroke_Scale_Score","eVitals_30":"Stroke_Scale_Type",
                        "eVitals_31":"Reperfusion_Checklist","eVitals_19":"Glasgow_Coma_Score_Eye","eVitals_20":"Glasgow_Coma_Score_Verbal",
                        "eVitals_21":"Glasgow_Coma_Score_Motor","ePatient_14":"Race","eHistory_01":"Barriers_to_Patient_Care",
                        "eMedications_03":"Date/Time_Medication_Administered","eMedications_05":"Medication_Dosage","eMedications_06":"Medication_Dosage_Units",
                        "eMedications_07":"Response_to_Medication","eMedications_10":"Role_of_Person_Administering_Medication",
                        "eMedications_02":"Medication_Administered_Prior_Unit_EMS_Care","eMedications_01":"Date/Time_Medication_Administered",
                        "eInjury_03":"Trauma_Center_Criteria"})

/var/folders/rl/dnlq62k50fd_dd8959xr62nh0000gn/T/ipykernel_19015/3629974227.py:1: DtypeWarning: Columns (77,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("DABP_Raw_data.csv")


In [3]:
df

,PcrKey,Disp_Complaint_Reported,EMD_Performed,Patient_Disposition,Final_Patient_Acuity,Transport_Mode,Destination,Hospital_Inpatient_Destination,Emergency_Department_Disposition,Hospital_Disposition,...,PcrMedicationKey,Date/Time_Medication_Administered,eMedications_03Descr,Medication_Dosage,Medication_Dosage_Units,Response_to_Medication,Role_of_Person_Administering_Medication,Medication_Administered_Prior_Unit_EMS_Care,Date/Time_Medication_Administered,Trauma_Center_Criteria
0,29723657,2301021,2302007,4212033,7701003,4216005,4221003,7701003,7701003,7701003,...,117368708,317361,EPINEPHrine 0.1 MG/ML,1.0,3706021,9916001,9905013,9923001,2019-01-16 12:28:34,7701003
1,29723657,2301021,2302007,4212033,7701003,4216005,4221003,7701003,7701003,7701003,...,117368708,317361,EPINEPHrine 0.1 MG/ML,1.0,3706021,9916001,9905013,9923001,2019-01-16 12:28:34,7701003
2,36585551,2301059,7701003,4212033,7701003,4216005,4221003,7701003,7701003,7701003,...,118882227,7701003,Not Recorded,0.0,7701003,7701003,7701003,7701003,Not Recorded,7701003
3,36585551,2301059,7701003,4212033,7701003,4216005,4221003,7701003,7701003,7701003,...,118882227,7701003,Not Recorded,0.0,7701003,7701003,7701003,7701003,Not Recorded,7701003
4,38930737,2301071,7701001,4212033,4219005,4216005,4221007,7701001,7701003,7701003,...,108893409,7701001,Not Applicable,0.0,7701001,7701001,7701001,7701001,Not Applicable,7701001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212637,80024875,2301061,2302003,4212033,4219005,4216005,4221003,7701001,7701003,7701003,...,125092049,161,Acetaminophen,1000.0,3706021,7701003,9905009,9923001,2019-08-12 13:05:00,7701003
1212638,80033457,2301021,7701003,4212033,7701003,4216005,4221003,7701001,7701003,7701003,...,125132750,4917,Nitroglycerin,0.4,3706021,9916003,7701003,9923001,2019-07-26 14:55:00,7701001
1212639,80033457,2301021,7701003,4212033,7701003,4216005,4221003,7701001,7701003,7701003,...,125132749,4917,Nitroglycerin,0.4,3706021,9916003,7701003,9923001,2019-07-26 14:42:00,7701001
1212640,80033457,2301021,7701003,4212033,7701003,4216005,4221003,7701001,7701003,7701003,...,125132750,4917,Nitroglycerin,0.4,3706021,9916003,7701003,9923001,2019-07-26 14:55:00,7701001


## Feature Selection
In this case, we are dealing with a lot of categorical variables with multiple interpretable meanings. Not much of the data can be interpreted numerically, and therefore we will have to either exclude it or develop our own numerical interpretation.

First is the outcome variable. We are trying to figure out what features are important for predicting patient death, therefore we can prioritize calls with those aspects when optimizing routing. We will use eoutcome 01 and eoutcome 02 for the outcome, which we will combine to form our y vector.

Keys with 20 indicates the patient has died either en route ot the hospital or while in the hospital's care, indicating it may be a situation where quicker response could have saved them. entries with 7701003 and 7701001 indicate that the column was not filed or not applicible, indicating no hospital transport. Therefore, these rows will be removed from our analysis.

In [4]:
ed_df = df.loc[(df['Emergency_Department_Disposition'] != 7701003) & (df['Emergency_Department_Disposition'] != 7701001)]

In [5]:
hosp_df = df.loc[(df['Hospital_Disposition'] != 7701003) & (df['Hospital_Disposition'] != 7701001)]

If a patient shows up in at least one of the two above dataframes, we know that they were transported to the hospial in the event

In [6]:
#Create index of all patients that were trainsported
transport_index = ed_df.index.union(hosp_df.index)

#
ML_df = df.iloc[transport_index]

In [7]:
ML_df = df.iloc[transport_index]

In [8]:
ML_df['patient_death'] = 0
ML_df.loc[(ML_df['Hospital_Disposition'] == 20) | (ML_df['Emergency_Department_Disposition'] == 20), 'patient_death'] = 1

/var/folders/rl/dnlq62k50fd_dd8959xr62nh0000gn/T/ipykernel_19015/4085509802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ML_df['patient_death'] = 0


In [9]:
# 579 positive samples, the data is very unbalanced
ML_df.loc[ML_df['patient_death']==1]

,PcrKey,Disp_Complaint_Reported,EMD_Performed,Patient_Disposition,Final_Patient_Acuity,Transport_Mode,Destination,Hospital_Inpatient_Destination,Emergency_Department_Disposition,Hospital_Disposition,...,Date/Time_Medication_Administered,eMedications_03Descr,Medication_Dosage,Medication_Dosage_Units,Response_to_Medication,Role_of_Person_Administering_Medication,Medication_Administered_Prior_Unit_EMS_Care,Date/Time_Medication_Administered,Trauma_Center_Criteria,patient_death
8348,74880607,2301071,2302001,4212033,7701003,4216005,4221005,4222035,7701003,20,...,7806,Oxygen,4.0,7701003,9916001,9905007,9923001,2019-12-22 12:04:00,7701003,1
8353,75042762,2301071,2302001,4212033,4219005,4216005,4221005,7701003,20,7701003,...,7701003,Not Recorded,0.0,7701003,7701003,7701003,7701003,Not Recorded,7701003,1
13501,64139813,2301061,7701003,4212033,7701003,4216005,4221003,7701001,9,20,...,703,Amiodarone,150.0,3706021,9916003,7701003,9923001,2019-11-10 14:40:00,7701003,1
19028,73122508,2301013,2302007,4212033,4219003,4216005,4221003,4222035,9,20,...,7806,Oxygen,4.0,3706035,9916001,9905007,7701003,2019-09-05 8:01:00,7701003,1
19472,62211091,2301019,2302007,4212033,4219001,4216005,4221005,4222035,20,20,...,7242,Naloxone,2.0,3706015,9916003,9905013,9923001,2019-10-24 20:17:00,7701001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189858,70230430,2301077,7701003,4212033,4219007,4216005,4221003,7701003,7701003,20,...,328316,Epinephrine 1 MG/ML,1.0,3706021,9916003,7701003,9923001,2019-12-11 7:44:00,7701003,1
1190190,75531087,2301041,2302001,4212033,4219001,4216005,4221003,7701003,9,20,...,317361,EPINEPHrine 0.1 MG/ML,1.0,3706021,9916003,9905013,9923001,2019-10-31 0:48:25,7701003,1
1203425,60495553,2301073,2302003,4212033,4219001,4216005,4221015,7701003,9,20,...,7806,Oxygen,10.0,3706035,7701003,9905007,7701003,2019-10-12 18:40:02,2903007,1
1205844,64919012,2301019,2302007,4212033,4219001,4216005,4221003,7701003,20,20,...,317361,EPINEPHrine 0.1 MG/ML,1.0,3706021,9916003,9905007,9923001,2019-11-10 11:21:00,7701003,1


In [10]:
ML_df['Date/Time_Medication_Administered']

,Date/Time_Medication_Administered,Date/Time_Medication_Administered
81,7701003,Not Recorded
283,7806,2019-11-28 19:46:41
338,7701003,Not Recorded
339,7701003,Not Recorded
390,7701003,Not Recorded
...,...,...
1212234,7701003,Not Recorded
1212288,7701003,Not Recorded
1212335,7701001,Not Applicable
1212380,7701003,Not Recorded


In [11]:
def process_column(df:pd.DataFrame, colname:str, categorical:bool, func=None, convert_NA:bool=False):

    null_like = [7701003, 7701001, ]
    
    #Assumes that columns are singular, if not, drop all but the first
    if len(df[colname].columns > 1):
        extra_cols = df.colname.columns[1:]
        df.drop(columns=extra_cols, inplace=True)

    if convert_NA:
        df[colname].replace(to_replace=null_like, value=pd.NA, inplace=True)

    if categorical:
        pd.get_dummies(df, columns=colname, )

    if func is not None:
        df[colname] = df[colname].apply(func, convertDType=True)

    return None

    
    

In [12]:
selected_cols = [
    'Alcohol/Drug_Use_Indicators', #Convert to binary drug invovlement score
    'Barriers_to_Patient_Care', #Raw categorical
    'Cause_of_Injury', #Raw categorical
    'Reason_for_Choosing_Destination', #Raw categorical
    # 'Date/Time_Medication_Administered', 
    'Primary_Symptom', #Raw categorical
    'Gender', #Raw categorical
    'Age', #Numerical, convert NA
    'Heart_Rate', #Numerical, convert NA
    'Pulse_Oximetry', #Numerical, convert NA
    'Stroke_Scale_Score',#Numerical, convert NA
    'Glasgow_Coma_Score_Eye', #Numerical, convert NA
    'Blood_Glucose_Level', #Numerical, convert NA
    'Type_of_Dispatch_Delay', #Convert to binary if delayed or not
    'Type_of_Response_Delay', #ditto
    'Type_of_Scene_Delay', #ditto
    'Type_of_Transport_Delay', #ditto
    # Combine the above to determine if delayed or not (add cutoff parameter?)
        # Will not be part of the function, break out that segment

]

In [13]:
ML_df = ML_df[selected_cols]

In [14]:
ML_df

,Alcohol/Drug_Use_Indicators,Barriers_to_Patient_Care,Cause_of_Injury,Reason_for_Choosing_Destination,Primary_Symptom,Gender,Age,Heart_Rate,Pulse_Oximetry,Stroke_Scale_Score,Glasgow_Coma_Score_Eye,Blood_Glucose_Level,Type_of_Dispatch_Delay,Type_of_Response_Delay,Type_of_Scene_Delay,Type_of_Transport_Delay
81,8801015,3101027,7701001,4220017,F03.91,9906003,75,53,94,7701003,4,7701003,2208013,2209011,7701003,7701003
283,7701003,3101009,7701001,4220001,R06.02,9906001,75,90,97,3329001,4,7701003,2208013,2209011,2210017,2211011
338,7701003,7701003,7701003,4220001,R41.82,9906001,85,7701003,7701003,3329001,7701003,7701003,7701003,2209011,2210017,2211011
339,7701003,7701003,7701003,4220001,R41.82,9906001,85,7701003,7701003,7701003,4,7701003,7701003,2209011,2210017,2211011
390,8801015,3101009,7701003,4220015,R41.82,9906001,59,90,99,3329001,4,215,2208013,2209011,2210017,2211011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212234,8801015,3101009,V49.9,4220001,M54.9,9906001,26,7701003,7701003,7701003,7701003,7701003,7701003,7701003,2210017,2211011
1212288,7701003,3101009,V03.1,4220021,R07.89,9906003,7,106,99,7701001,4,7701003,2208013,2209011,2210017,2211011
1212335,7701001,3101027,7701001,4220021,R06.02,9906003,67,125,98,7701001,1,7701001,7701003,2209011,2210025,2211011
1212380,8801015,3101009,7701003,4220021,R46.3,9906003,29,76,96,7701003,4,7701003,2208013,2209011,2210017,7701003


In [15]:
categoricals = [
    'Barriers_to_Patient_Care', #Raw categorical
    'Cause_of_Injury', #Raw categorical
    'Reason_for_Choosing_Destination', #Raw categorical
    'Primary_Symptom', #Raw categorical
    'Gender', #Raw categorical
]

numericals = [
    'Age', #Numerical, convert NA
    'Heart_Rate', #Numerical, convert NA
    'Pulse_Oximetry', #Numerical, convert NA
    'Stroke_Scale_Score',#Numerical, convert NA
    'Glasgow_Coma_Score_Eye', #Numerical, convert NA
    'Blood_Glucose_Level', #Numerical, convert NA 
]

modified = [
    'Alcohol/Drug_Use_Indicators', #Convert to binary drug invovlement score
    'Type_of_Dispatch_Delay', #Convert to binary if delayed or not
    'Type_of_Response_Delay', #ditto
    'Type_of_Scene_Delay', #ditto
    'Type_of_Transport_Delay', #ditto
]

In [16]:
#Convert categoricals

def convert_cats(df_in:pd.DataFrame, catlist:list):

    df = df_in.copy()

    #Ensure the column only shows up once, take the first one otherwize
    for colname in catlist:
        if type(df[colname]) != pd.Series:
            if len(df[colname].columns > 1):
                extra_cols = df.colname.columns[1:]
                df.drop(columns=extra_cols, inplace=True)

    cats_unrolled = pd.get_dummies(df, columns=categoricals)
    df = df.drop(columns=categoricals)

    return pd.concat([df, cats_unrolled], axis=1)


ML_df = convert_cats(ML_df, categoricals)

In [19]:
null_like = [7701003, 7701001, ]


def convert_numerical(df_in:pd.DataFrame, collist:list, null_like:list):

    df = df_in.copy()

    for colname in collist:
        #Remove extra columns
        if type(df[colname]) != pd.Series:
            if len(df[colname].columns) > 1:
                extra_cols = df[colname].columns[1:]
                df.drop(columns=extra_cols, inplace=True)

        
        col = df[colname].replace(to_replace=null_like, value=pd.NA, inplace=True)
        col = pd.to_numeric(col, errors='coerce')

        df[colname] = col

    return df


ML_df = convert_numerical(ML_df, numericals, null_like=null_like)

KeyError: 'Age'